In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')
from tqdm import tqdm


In [4]:
user_question = "I just discovered the course. Can I still join it?"
len(embedding_model.encode(user_question))

768

Q1. Getting the embeddings model

What's the first value of the resulting vector?

In [5]:
embedding_model.encode(user_question)[0]

0.07822266

Prepare the documents
Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

In [7]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

In [9]:
len(documents)

948

In [12]:
mlops_documents = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]

In [13]:
len(mlops_documents)

375

Q2. Creating the embeddings
Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

Create a list embeddings
Iterate over each document
qa_text = f'{question} {text}'
compute the embedding for qa_text, append to embeddings
At the end, let X = np.array(embeddings) (import numpy as np)
What's the shape of X? (X.shape). Include the parantheses.

In [26]:
#created the dense vector using the pre-trained model
emeddings = []
for doc in tqdm(mlops_documents):
    # Transforming the title into an embedding using the model
    vector = embedding_model.encode(f'{doc["question"]} {doc["text"]}').tolist()
    emeddings.append(vector)

In [ ]:
import numpy as np
X = np.array(emeddings)

In [ ]:
X.shape

Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:

In [ ]:
from numpy.linalg import norm

In [ ]:
def cosine_distance(vector, matrix, normalized = False):
    
    
    if normalized == False:
        # Нормализация вектора
        vector = vector / np.linalg.norm(vector)

        # Нормализация матрицы (каждой строки)
        matrix = matrix / np.linalg.norm(matrix, axis=1)[:, np.newaxis]
    
    # Вычисление косинусного расстояния (1 - косинусное сходство)
    cosine_similarity = np.dot(matrix, vector)
    cosine_distance = 1 - cosine_similarity
    
    return cosine_distance

# Пример использования
vector = np.array([1, 2, 3])
matrix = np.array([
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])

distances = cosine_distance(vector, matrix, normalized= False)
print(distances)

In [ ]:
v = embedding_model.encode(user_question)
distances = cosine_distance(v, X, normalized = True)
max(distances)

In [ ]:
scores = X.dot(v)

In [ ]:
max(scores)

Vector search
We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=mlops_documents, embeddings=X)
search_engine.search(v, num_results=5)

In [ ]:
len(search_engine.search(v, num_results=5))

Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

First, load the ground truth dataset:

In [ ]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
[doc for doc in ground_truth if doc['document'] =='0a278fb2']

In [ ]:
hits = []
for doc in tqdm(ground_truth):
    v = embedding_model.encode(f'{doc["question"]}').tolist()
    results = search_engine.search(v, num_results=5)
    ids = [res['id'] for res in results]
    if doc['document'] in ids:
        hits.append(1)
    else:
        hits.append(0)

In [ ]:
np.mean(hits)

Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

Create the index with the same settings as in the module (but change the dimensions)
Index the embeddings (note: you've already computed them)
After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

In [ ]:

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(mlops_documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

In [ ]:
len(results)

In [ ]:
elastic_indexing = []
for doc in tqdm(mlops_documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text
    
    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)
    elastic_indexing.append(doc)

In [ ]:
for doc in tqdm(elastic_indexing):
    es_client.index(index=index_name, document=doc)

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },

            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
_ = elastic_search(
    query=user_question,
    #course="data-engineering-zoomcamp"
)

In [ ]:
_[0]['id']

In [108]:
np.mean(hits_elastic)

0.7530054644808744